In [27]:
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
from google.colab import files
import io

In [28]:
uploaded = files.upload()
data = np.genfromtxt(io.BytesIO(uploaded['perceptron_toydata.txt']))

Saving perceptron_toydata.txt to perceptron_toydata (3).txt


In [29]:
X = torch.tensor(data[:,:2], dtype=torch.float32)
y = torch.tensor(data[:,2], dtype=torch.int64)

torch.manual_seed(123)
shuffle_idx = torch.randperm(y.size(0), dtype=torch.long)

X, y = X[shuffle_idx], y[shuffle_idx]

percent75 = int(shuffle_idx.size(0)*0.75)

X_train, X_test = X[shuffle_idx[:percent75]], X[shuffle_idx[percent75:]]
y_train, y_test = y[shuffle_idx[:percent75]], y[shuffle_idx[percent75:]]

# Normalizing
mu, sigma = X_train.mean(dim=0), X_train.std(dim=0)
X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma

In [30]:
class LogisticRegression2(torch.nn.Module):
  def __init__(self, num_features):
    super(LogisticRegression2, self).__init__()
    self.linear = torch.nn.Linear(num_features, 1)
    self.linear.weight.detach().zero_()
    self.linear.bias.detach().zero_()
  
  def forward(self, x):
    return torch.sigmoid(self.linear(x))

In [31]:
def custom_where(cond, x_1, x_2):
  return (cond * x_1) + ((1 - cond) * x_2)
def comp_accuracy(y, activation):
  pred_labels = custom_where((activation > 0.5).float(), 1, 0).view(-1)
  return torch.sum(pred_labels == y.view(-1)).float() / y.size(0)

In [32]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = LogisticRegression2(num_features=2).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=2)

num_epochs = 30
X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device).view(-1, 1)

<ipython-input-32-757ff3617fe0>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
<ipython-input-32-757ff3617fe0>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device).view(-1, 1)


In [33]:
for epoch in range(num_epochs):
  # Compute outputs
  out = model(X_train_tensor)
  # Compute gradients
  cost = F.binary_cross_entropy(out, y_train_tensor, reduction='sum')
  optimizer.zero_grad()
  cost.backward()
  # Update weights
  optimizer.step()
  # Logging
  activation = model(X_train_tensor)
  acc = comp_accuracy(y_train_tensor, activation)
  print('Epoch: %03d' % (epoch + 1), end='')
  print(' | Train aCC: %.3f' % acc, end='')
  print(' | Cost: %.3f' % F.binary_cross_entropy(activation, y_train_tensor))

print('\nModel parameters:')
print('  Weights: %s' % model.linear.weight)
print('  Bias: %s' % model.linear.bias)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)

activation = model(X_test_tensor)
test_acc = comp_accuracy(y_test_tensor, activation)
print('\n\nTest set accuracy: %.2f%%' % (test_acc * 100))

Epoch: 001 | Train aCC: 0.973 | Cost: 0.066
Epoch: 002 | Train aCC: 0.973 | Cost: 0.071
Epoch: 003 | Train aCC: 0.973 | Cost: 0.085
Epoch: 004 | Train aCC: 0.973 | Cost: 0.099
Epoch: 005 | Train aCC: 0.973 | Cost: 0.108
Epoch: 006 | Train aCC: 0.973 | Cost: 0.112
Epoch: 007 | Train aCC: 0.973 | Cost: 0.114
Epoch: 008 | Train aCC: 0.973 | Cost: 0.115
Epoch: 009 | Train aCC: 0.973 | Cost: 0.115
Epoch: 010 | Train aCC: 0.973 | Cost: 0.115
Epoch: 011 | Train aCC: 0.973 | Cost: 0.116
Epoch: 012 | Train aCC: 0.973 | Cost: 0.116
Epoch: 013 | Train aCC: 0.973 | Cost: 0.116
Epoch: 014 | Train aCC: 0.973 | Cost: 0.116
Epoch: 015 | Train aCC: 0.973 | Cost: 0.116
Epoch: 016 | Train aCC: 0.973 | Cost: 0.116
Epoch: 017 | Train aCC: 0.973 | Cost: 0.116
Epoch: 018 | Train aCC: 0.973 | Cost: 0.116
Epoch: 019 | Train aCC: 0.973 | Cost: 0.116
Epoch: 020 | Train aCC: 0.973 | Cost: 0.116
Epoch: 021 | Train aCC: 0.973 | Cost: 0.116
Epoch: 022 | Train aCC: 0.973 | Cost: 0.116
Epoch: 023 | Train aCC: 0.973 | 

<ipython-input-33-9a3375dee2a1>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
<ipython-input-33-9a3375dee2a1>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)
